In [1]:
#Import Libraries

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import random
import numpy as np
import torch
from torch.autograd import Variable
import requests

# Parse fasta file and create training data

In [2]:
# Load Fasta with biopython parser
from Bio import SeqIO
import pandas as pd
# Create list for pos training data
my_seqlist = []
for seq_record in SeqIO.parse(open('/iblm/netapp/data1/jjaureguy/HiChip_project/HiChip_peak_results/GSE168881/SRR13961069.fa.out'),'fasta'):
    my_seqlist.append(seq_record)
my_seqlist[0].seq

print(my_seqlist[0])
print(len(my_seqlist))



FileNotFoundError: [Errno 2] No such file or directory: '/iblm/netapp/data1/jjaureguy/HiChip_project/HiChip_peak_results/GSE168881/SRR13961069.fa.out'

In [3]:
bases = 'ACTG'
base_map = dict(zip(bases, range(len(bases))))
n_classes = len(base_map)
base_map, n_classes

({'A': 0, 'C': 1, 'T': 2, 'G': 3}, 4)

In [4]:
np.random.seed(42)

In [5]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [6]:
pos_seqlist = []
neg_seqlist = []

skipped = 0
for i in my_seqlist:
    fasta = i.seq
    as_text = fasta.upper()
    if 'N' in as_text:
        skipped += 1
    else:
        seq = np.vectorize(base_map.get)(np.array(list(as_text)))
        pos_seqlist.append(seq)
        neg_seqlist.append(np.random.choice(seq, size=len(seq), replace=False))
    
seqlist = np.stack(pos_seqlist + neg_seqlist)
#seqlist = list(filter(None, seqlist))  # This removes empty sequences.
n_samples = len(seqlist)
n_samples, skipped


ValueError: need at least one array to stack

In [7]:
training_data = one_hot(seqlist, n_classes).reshape(n_samples, -1, n_classes)
training_data.shape

NameError: name 'seqlist' is not defined

# Log regression var

In [8]:
training_data.reshape(n_samples, -1).shape
x_log = training_data.reshape(n_samples, -1)
x_log.shape

NameError: name 'training_data' is not defined

# Create labels(response variable Y)

In [ ]:
response_var_pos = [1]*(n_samples//2)
response_var_neg = [0]*(n_samples//2)

total_response_var = response_var_pos + response_var_neg
print(len(total_response_var))
print(total_response_var[7282:7285])
total_response_var = np.array(total_response_var)
total_response_var.shape

# One Hot Encode Class(Extra)


In [ ]:
class hot_dna:
    def __init__(self,fasta):
        #check for and grab sequence name
        name = 'unknown_sequence'
        sequence = fasta
        #get sequence into an array
        seq_array = array(list(sequence))
        label_encoder = LabelEncoder()
        integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
        #one hot the sequence
        onehot_encoder = OneHotEncoder(sparse=False)
        #reshape because that's what OneHotEncoder likes
        integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
        onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

        #add the attributes to self 
        self.name = name
        self.sequence = fasta
        self.integer = integer_encoded_seq
        self.onehot = onehot_encoded_seq

# One hot encode training data

In [ ]:
#One hot encode training data 
one_hot_enc_train = []
for i in seqlist:
    hot_enc = hot_dna(i)
    #print(hot_enc.integer)
    one_hot_enc_train.append(hot_enc.onehot)
#one_hot_enc[0]


print(len(one_hot_enc_train))
print(one_hot_enc_train[0].shape)

In [ ]:
shapes = set()
for seq in one_hot_enc_train:
    shapes.add(seq.shape)

In [ ]:
shapes

In [ ]:

one_hot_enc_train = np.stack(one_hot_enc_train)
#one_hot_enc_train.toarray()
print(len(one_hot_enc_train))

print(one_hot_enc_train.shape)

In [ ]:
one_hot_enc_train[0].shape

# Split data into train and test sets

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# The LabelEncoder encodes a sequence of bases as a sequence of integers.
integer_encoder = LabelEncoder()  
# The OneHotEncoder converts an array of integers to a sparse matrix where 
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')   
# Positive training list
input_features = []


for sequence in total_train_data:
    #print(sequence)
    #fasta = sequence.seq
    integer_encoded = integer_encoder.fit_transform(sequence)
    #print(integer_encoded)
    ##########RESHAPE issue##########(14,000,-1,1)
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features.append(one_hot_encoded.toarray())

np.set_printoptions(threshold=40)
input_features = np.stack(input_features)

#input_features = np.array(input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',total_train_data[0][:10],'...',total_train_data[0][-10:])
print('One hot encoding of Sequence #1:\n',input_features[0].T)


print(input_features.ndim)
print(input_features.shape)

In [ ]:
#check dimensions of arrays
#one_hot_enc_train.ndim

print(one_hot_enc_train[1])
#Check input_features
one_hot_enc_train = np.hstack(one_hot_enc_train)

one_hot_enc_train = np.array(one_hot_enc_train)
print('training data dimension:',one_hot_enc_train.ndim)
print(('training data shape: ',one_hot_enc_train.shape))

print("X var dimension:",input_features.ndim)
print("X var dimension:",input_features.shape)

#Check input_labels(response variables)
print('Y var dimension: ',input_labels.ndim)
print(('Y var shape:',input_labels[0].shape))
#print(len(input_labels))

# Additional training exploration

In [ ]:
random.seed(42)

#random.shuffle(one_hot_enc)
#prediction = list(my_seqlist.reverse)
prediction_list = list(reversed(my_seqlist))
print(len(prediction_list))

# Test one hot enc

In [ ]:
one_hot_enc = []
for i in my_seqlist:
    fasta = i.seq
    #print(fasta[:10])
    hot_enc = hot_dna(fasta)
    #print(hot_enc.integer)
    one_hot_enc.append(hot_enc.onehot)
#one_hot_enc[0]



In [ ]:
print(len(one_hot_enc))
print(one_hot_enc[0])

In [ ]:
neg_train[0].seq

In [ ]:
one_hot_enc_neg = []
for i in neg_train:
    fasta = i.seq
    hot_enc = hot_dna(fasta)
    #print(hot_enc.integer)
    one_hot_enc_neg.append(hot_enc.onehot)
#one_hot_enc[0]

In [ ]:
one_hot_enc_neg[0]

In [ ]:
print(len(one_hot_enc_neg))

# One hot encode training data(alternative approach from tutorial) no producing correct dims

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# The LabelEncoder encodes a sequence of bases as a sequence of integers.
integer_encoder = LabelEncoder()  
# The OneHotEncoder converts an array of integers to a sparse matrix where 
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')   
# Positive training list
input_features = []


for sequence in total_train_data:
    fasta = sequence.seq
    integer_encoded = integer_encoder.fit_transform(list(fasta))
    #print(integer_encoded)
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features.append(one_hot_encoded.toarray())

np.set_printoptions(threshold=40)
input_features = np.hstack(input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',total_tain_data[0][:10].seq,'...',total_tain_data[0][-10:].seq)
print('One hot encoding of Sequence #1:\n',input_features[0].T)


print(len(input_features))

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# The LabelEncoder encodes a sequence of bases as a sequence of integers.
integer_encoder = LabelEncoder()  
# The OneHotEncoder converts an array of integers to a sparse matrix where 
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')   
# Positive training list
input_features = []
# Negative training list
input_features_neg = []
# Prediction reversed one hot encode
input_features_prediction_list = []
#Postive training list one hot encode

for sequence in my_seqlist:
    fasta = sequence.seq
    integer_encoded = integer_encoder.fit_transform((fasta))
    #print(integer_encoded)
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features.append(one_hot_encoded.toarray())

np.set_printoptions(threshold=40)
input_features = np.hstack(input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',my_seqlist[0][:10].seq,'...',my_seqlist[0][-10:].seq)
print('One hot encoding of Sequence #1:\n',input_features[0].T)

#Negative training list one hot encode

for sequence in neg_train:
    fasta = sequence.seq
    integer_encoded = integer_encoder.fit_transform((fasta))
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features_neg.append(one_hot_encoded.toarray())

np.set_printoptions(threshold=40)
input_features_neg = np.hstack(input_features_neg)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',neg_train[0][:10].seq,'...',neg_train[0][-10:].seq)
print('One hot encoding of Sequence #1:\n',input_features_neg[0].T)


# #Prediction reversed one hot encode
# for sequence in prediction_list:
#     fasta = sequence.seq
#     integer_encoded = integer_encoder.fit_transform((fasta))
#     integer_encoded = np.array(integer_encoded).reshape(-1, 1)
#     one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
#     input_features_prediction_list.append(one_hot_encoded.toarray())

# np.set_printoptions(threshold=40)
# input_features_prediction_list = np.hstack(input_features_prediction_list)
# print("Example sequence\n-----------------------")
# print('DNA Sequence #1:\n',prediction_list[0][:10].seq,'...',prediction_list[0][-10:].seq)
# print('One hot encoding of Sequence #1:\n',input_features_prediction_list[0].T)



# Logistic Regression model

#Split data into a train/test split

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

print(x_log.shape)
print(total_response_var.shape)

X_train, X_test, y_train, y_test = train_test_split(x_log, total_response_var, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

#Build the pytorch logistic model

In [ ]:
import torch
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [ ]:
#Initialize model




epochs = 10000
input_dim = 4000  # Two inputs x1 and x2 
output_dim = 1 # Single binary output 
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_train.requires_grad = True
X_test = torch.from_numpy(X_test.astype(np.float32))
X_test.requires_grad = True
y_train = torch.from_numpy(y_train.astype(np.float32))
y_train.requires_grad = True
y_test = torch.from_numpy(y_test.astype(np.float32))
y_test.requires_grad = True

In [ ]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [ ]:
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    #backward pass
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    optimizer.zero_grad()
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')

# Linear regression 

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt


In [ ]:
#Initialize model




epochs = 10000
input_dim = 4000  # Two inputs x1 and x2 
output_dim = 1 # Single binary output 
learning_rate = 0.01

#model
model = nn.Linear(input_dim,output_dim)
#Loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [ ]:
#Loop
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    #backward pass
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    optimizer.zero_grad()
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')

        
        

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')

# CNN 


In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
training_data.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_log, total_response_var, test_size=0.1, random_state=42)

In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_train.requires_grad = True
X_test = torch.from_numpy(X_test.astype(np.float32))
X_test.requires_grad = True
y_train = torch.from_numpy(y_train.astype(np.float32))
y_train.requires_grad = True
y_test = torch.from_numpy(y_test.astype(np.float32))
y_test.requires_grad = True

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#hyper parameters
input_size = 1000*4
hidden_size = 2000
classes = 1
epochs = 140
learning_rate = 0.01

In [ ]:
#defining the network
from torch import nn
from torch.nn import functional as F
class test_net(nn.Module):
    def __init__(self):
        super(test_net,self).__init__()
        # self.conv1 = nn.Conv2d(i)
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size,1000)
        self.fc3 = nn.Linear(1000,classes)
    def forward(self,X):
        # out = out.flatten()
        out = self.fc1(X)
        out = torch.tanh(out)
        out = self.fc2(out)
        out = torch.tanh(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)
        return out

In [ ]:

#model
model = test_net()
#Loss and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
#Loop
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    
    
    #forward pass and loss
    y_predicted = model(X_train)
    # print(y_predicted)
    loss = criterion(y_predicted, y_train)
    # print(loss)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')
        y_test_preds = model(X_test)
        test_loss = criterion(y_test_preds, y_test)
        print('test loss: ' + str(test_loss))

        
        

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')
    # test_loss = criterion(y_predicted, y_test)

# Fully connected Neural net

In [ ]:
# Hyper-parameters 
input_size = 4000
hidden_size = 2000
num_classes = 1
epochs = 1000
# batch_size = 100
learning_rate = 0.001

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = torch.sigmoid(out)
        return out

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
# Loss and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
#Loop
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    
    
    #forward pass and loss
    y_predicted = model(X_train)
    # print(y_predicted)
    loss = criterion(y_predicted, y_train)
    # print(loss)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')
        y_test_preds = model(X_test)
        test_loss = criterion(y_test_preds, y_test)
        print('test loss: ' + str(test_loss))

        
        

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')
    # test_loss = criterion(y_predicted, y_test)